In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.2'

# LSTM으로 텍스트 생성하기

이 노트북은 [케라스 창시자에게 배우는 딥러닝](https://tensorflow.blog/%EC%BC%80%EB%9D%BC%EC%8A%A4-%EB%94%A5%EB%9F%AC%EB%8B%9D/) 책의 8장 1절의 코드 예제입니다. 책에는 더 많은 내용과 그림이 있습니다. 이 노트북에는 소스 코드에 관련된 설명만 포함합니다.

----

[...]

## 글자 수준의 LSTM 텍스트 생성 모델 구현

이런 아이디어를 케라스로 구현해 보죠. 먼저 언어 모델을 학습하기 위해 많은 텍스트 데이터가 필요합니다. 위키피디아나 반지의 제왕처럼 아주 큰 텍스트 파일이나 텍스트 파일의 묶음을 사용할 수 있습니다. 이 예에서는 19세기 후반 독일의 철학자 니체의 글을 사용하겠습니다(영어로 번역된 글입니다). 학습할 언어 모델은 일반적인 영어 모델이 아니라 니체의 문체와 특정 주제를 따르는 모델일 것입니다.

## 데이터 전처리

먼저 말뭉치를 다운로드하고 소문자로 바꿉니다:

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

말뭉치 크기: 600893


In [3]:
type(text)

str

그 다음 `maxlen` 길이를 가진 시퀀스를 중복하여 추출합니다. 추출된 시퀀스를 원-핫 인코딩으로 변환하고 크기가 `(sequences, maxlen, unique_characters)`인 3D 넘파이 배열 `x`로 합칩니다. 동시에 훈련 샘플에 상응하는 타깃을 담은 배열 `y`를 준비합니다. 타깃은 추출된 시퀀스 다음에 오는 원-핫 인코딩된 글자입니다.

In [4]:
# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60

# 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링합니다.
step = 3

# 추출한 시퀀스를 담을 리스트
sentences = []

# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 57
벡터화...


## 네트워크 구성

이 네트워크는 하나의 `LSTM` 층과 그 뒤에 `Dense` 분류기가 뒤따릅니다. 분류기는 가능한 모든 글자에 대한 소프트맥스 출력을 만듭니다. 순환 신경망이 시퀀스 데이터를 생성하는 유일한 방법은 아닙니다. 최근에는 1D 컨브넷도 이런 작업에 아주 잘 들어 맞는다는 것이 밝혀졌습니다.

In [5]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

타깃이 원-핫 인코딩되어 있기 때문에 모델을 훈련하기 위해 `categorical_crossentropy` 손실을 사용합니다:

In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## 언어 모델 훈련과 샘플링

훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

1.	지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
2.	특정 온도로 이 확률 분포의 가중치를 조정합니다.
3.	가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
4.	새로운 글자를 생성된 텍스트의 끝에 추가합니다.

다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다(샘플링 함수입니다):

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

마지막으로 다음 반복문은 반복적으로 훈련하고 텍스트를 생성합니다. 에포크마다 학습이 끝난 후 여러가지 온도를 사용해 텍스트를 생성합니다. 이렇게 하면 모델이 수렴하면서 생성된 텍스트가 어떻게 진화하는지 볼 수 있습니다. 온도가 샘플링 전략에 미치는 영향도 보여 줍니다.

In [8]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 60 에포크 동안 모델을 훈련합니다
for epoch in range(1, 60):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
Epoch 1/1
200278/200278 [==============================] - 78s 390us/step - loss: 1.9812
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the resting in the still to the spirity of the resting and that is and and that has and that is intermand and that the stare and some and and and properses that is a so and that the resting that who has and that the resting in the restination of the stares and that is and the stare of the restination of the will and the resting and that the senses and the resting of the stare of the spirity that
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the "there so for and fang, in such and so parhy can that is it is and in the something that his somewhish propes to so and the diling of consequent procoest there
so its as a song, without ranger mistorally and dispossible and and and the them and and the remations and that i

wishness. one carrisiof. of to 
opposing," save anded.

geng of
the circumstanceni. many unword every became, a greary
prindent and and bark beachinat
an
cerelence, a naturally. one mode. this in ser, nyises of athensadne, to pand for the heart
and me as i   i
에포크 9
Epoch 1/1
200278/200278 [==============================] - 78s 390us/step - loss: 1.3973
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the same a strength of the most more and the soul the fact the condition of the man and the belief the man and all the precisely that in the condition of the man and the sense of the condition of the soul interest of the present the man and the soul and the will to be so the same the sour. the man as a condition of the present and the present the soul in the present and the distinguished and the
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the intellect--w

are, howeys.=--what
have .ke new umbses, pann of 
------ 온도: 1.2
the slowly ascending ranks and classes, in which,
through for which a same intoniigiar to eyes
have under forems, them basist,
have gains reconding of medoion. on
benefice to
instaed
mere in god, simply whom inburafice in gakess, too,--which fighterising without matter give
they not looks an hidman point" of the spiritualiea impulsed that
rank, there
tousm
in
sensution ented, the the harm mediocret in
where for
powerful world that ewver be preclused rather
에포크 17
Epoch 1/1
200278/200278 [==============================] - 78s 391us/step - loss: 1.34551
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the same as a senses and the conditions of the fact, and the and something the presence of the fact the same and the respect of the same that is also the problem of the conditions of the same the the senses of the acts and so

/home/haesun/anaconda3/envs/deep-learning-with-python/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 sense of the strength and stritter of the same things and the same almost answer a process and an ancient greek the sense of the contemplation of the strength and stritter of the same things and the same things, and the sense of t
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the most which the enemy, and said the voluntary and
sense themselves so the conclusion of them to a gained to men, or in fact that the establish development of restriated purely effective in the spirit and strictly and the man as a recollective of them and signal reason in the most conduct there is a south is an inculation of the casestie the same things with the sign of
the later the religious
------ 온도: 1.0
the slowly ascending ranks and classes, in which,
through for the spirit and science to great work in regard to conscience in its infulletentus, there is even exascail modern interbest,
when they all regulser sighter of a moreabned germans at lear interpretations of
all there 

...

through for the presence of the consequently and the state of the character and morality and accordinary to the state of the consequently of the sense of the constant sense of the state of the free spirits and the most morality of the consequently of the sense of the soul is a man is a man who contempt of the sense of the strength and stronger and always because our self-delighted and the present a conseque
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the thing man in one will precisely the "most satisfaction to an antimally that the instincts and decay they strong the dest of the enement of ordeal of actions of presented in the instance of the case of the soul is not a great profound, as in the most expression of characteristic of the delicate the interroe the fearfulness has to have been assing of the faith in the instinct and interpretatio
------ 온도: 1.0
the slowly ascending ranks and classes, in which,
through for therefore mere, he lurghed in our ac

through for the suffered and the suffering in the same there is the distrust of the spirit of the same the and the highest such a soul, the highest such an intermous and such and something of the suffered to the same a sort of properting to the sense of the sense of the same as they are in the spirit of the sense of the sufferer of the soul and the science and the superiority of the sense of the sense of th
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for instance of the since of the spiriatic suffering him is they soul, the
restracts of the can give the promise of the and the end of profounds and
personal uncondition of many with the free spirit of a power, disthrough such a god. they are as the distrust, and they are hence the same the world of the stronger
sympathy to the inclable to religions as the world and they and the same as such her far
------ 온도: 1.0
the slowly ascending ranks and classes, in which,
through for good and appear to movef "free siffe

through for the strongly and the most souls of the fact that the delicate the superiority of the state of the strong of the same only that the end and in the state of the state of the state and the sense of the most and the manifest of the states of the free of the superiority of the stronger and the proper present mediocre, in the enemy and superiority of the strong of the state of the sense of the most so
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the soul and such a new a denualis, of the most the states of the discovered and the fact to the contempt of them was the most conscience of the moral and some even in us the charming of the same as the man
which should seems to the most acts of herating even of the most men as resists the stronger for the end of entire green of all the senses will "free spirits must be it is all the state of an
------ 온도: 1.0
the slowly ascending ranks and classes, in which,
through for said. the
talses; as it
very ranks a

through for the superfacious of the subtle
truth of the same thing that the soul is not for the superstion of the the same in the superstion of the consens and consequence and the profounding of the constant the superstion of the faith of the sublime and moral the enjoywnds, and the development of the being the best of the the conserved of the sublime of the faith and the moral the moral the contrary the su
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through formed of the developed them a complained and the soul-a which is the way to every anything something in the antide in the beine. the perfect of the spirit is not well freeas them
thought and conded to an above all which therefore is the scend of truth that the words of the purely also which the missticklant and and soul of the best the nations of the person whoc fautted to the all the great moral d
------ 온도: 1.0
the slowly ascending ranks and classes, in which,
through foreance,
who, thou seems.= and philosop

through for the same allied with the same allious, which the profound be some the fact that the most conditions of the fact that the self-also the same the same the constangers, and the profound and the respect to the condition of the world and the world and the self-an according to the same things and the age and the profound and the person and the profound and the profound be some the fact that is the sam
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through former of the points the things some doom to be the fact on the end-also in the basis of the point of the schopenhwaring of the power of the will to be the hand, in the precisely even the person of the "good
are of the most duty distingus and the feathing is not the distinguished to
the subject so himself the operates as the conduct and things, the over
the exaggerity have a modes and that their ten
------ 온도: 1.0
the slowly ascending ranks and classes, in which,
through for it nose of morality, cateme; the the

여기서 볼 수 있듯이 낮은 온도는 아주 반복적이고 예상되는 텍스트를 만듭니다. 하지만 국부적인 구조는 매우 실제와 같습니다. 특히 모든 단어(단어는 글자의 지역 패턴으로 이루어집니다)가 실제 영어 단어입니다. 높은 온도에서 생성된 텍스트는 아주 흥미롭고 놀라우며 창의적이기도 합니다. 이따금 꽤 그럴싸하게 보이는 완전히 새로운 단어를 창조합니다(‘begarmed’와 ‘isharent’ 같은 단어입니다). 높은 온도에서는 국부적인 구조가 무너지기 시작합니다. 대부분의 단어가 어느정도 무작위한 문자열로 보입니다. 확실히 이 네트워크에서는 텍스트 생성에 가장 좋은 온도는 0.5입니다. 항상 다양한 샘플링 전략으로 실험해 봐야합니다! 학습된 구조와 무작위성 사이에 균형을 잘 맞추면 흥미로운 것을 만들 수 있습니다.

더 많은 데이터에서 크고 깊은 모델을 훈련하면 이것보다 훨씬 논리적이고 실제와 같은 텍스트 샘플을 생성할 수 있습니다. 당연히 우연이 아닌 의미 있는 텍스트가 생성된다고 기대하지 마세요. 글자를 연속해서 나열하기 위한 통계 모델에서 데이터를 샘플링한 것뿐입니다. 언어는 의사소통의 수단입니다. 의사소통이 의미하는 것과 의사소통이 인코딩된 메시지의 통계 구조 사이는 차이가 있습니다. 이 차이를 검증하기 위해 다음과 같은 사고 실험을 해보죠. 컴퓨터가 대부분의 디지털 통신에서 하는 것처럼 사람의 언어가 의사소통을 압축하는데 더 뛰어나다면 어떨까요? 언어의 의미가 줄진 않지만 고유한 통계 구조가 사라질 것입니다. 이는 방금과 같은 언어 모델을 학습하는 것을 불가능하게 만듭니다.

## 정리

* 이전의 토큰이 주어지면 다음 토큰(들)을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있습니다.
* 텍스트의 경우 이런 모델을 언어 모델이라 부릅니다. 단어 또는 글자 단위 모두 가능합니다.
* 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 합니다.
* 이를 위해 소프트맥스 온도 개념을 사용합니다. 항상 다양한 온도를 실험해서 적절한 값을 찾습니다.